<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller:  Recomendando Música

El objetivo de este caso-taller es construir un sistema de recomendación de Música utilizando los datos de [Last.fm](https://www.last.fm/) provistos  abiertamente por [grouplens](https://grouplens.org/about/what-is-grouplens/) para: **"avanzar la teoría y la práctica de la computación social mediante la construcción y la comprensión de sistemas *(de recomendación)* utilizados por personas reales".**

Los datos contienen información sobre artistas, usuarios, y las veces que estos escucharon sus canciones. Las bases se encuentran en los `Archivos de Laboratorio` en la carpeta `data`, allí también está disponible un archivo [README](data/readme.txt) que contiene más información sobre las bases.


## Instrucciones generales

1. Para desarrollar el *cuaderno*, primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en el mismo `path` de su cuaderno, por ejemplo: `data`.

## Desarrollo


### 1. Carga de datos 

En la carpeta `data` se encuentran los archivos:

   - `artists.dat`  que contienen el identificador del artista (`id`), nombre (`name`), link a la página del artista en last.fm (`url`), y link a la imagen del usuario (`pictureURL`), vale aclarar que varios de estos links están rotos. 
   - `user_artists.dat`  que contiene identificador del usuario (`userID`), nombre del artista que escuchó (`artistID`), y las veces que los escuchó (`weight`).

Cargue estos datos en su *cuaderno*:

   1. Para la base de artistas seleccione las columnas de identificador de artista (`id`) y nombre (`name`). Renombre estas columnas para poder hacer la unión con la base `user_artists.dat`.
   2. Para la base de usuarios y artistas, renombre las columnas de forma tal que se mantenga la consistencia para unir con la base anterior, y renombre la columna `weight` a `nro_reproducciones`.
   3. Una estas bases.
   

In [1]:
#! pip install plotly
#! pip install nbformat>=4.2.0
#! pip install ipywidgets

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
import pandas as pd
import plotly.express as px

In [3]:
# Utilice este espacio para escribir el código.
artists = pd.read_csv('data/artists.dat', sep='\t', usecols=[0, 1], header=0, names=['artistID', 'artistName'])
user_artists = pd.read_csv('data/user_artists.dat', sep='\t', header=0, names=['userID', 'artistID', 'nro_reproducciones'])

users_artists = pd.merge(artists, user_artists, on='artistID')
users_artists

,artistID,artistName,userID,nro_reproducciones
0,1,MALICE MIZER,34,212
1,1,MALICE MIZER,274,483
2,1,MALICE MIZER,785,76
3,2,Diary of Dreams,135,1021
4,2,Diary of Dreams,257,152
...,...,...,...,...
92829,18741,Diamanda Galás,454,301
92830,18742,Aya RL,454,294
92831,18743,Coptic Rain,454,287
92832,18744,Oz Alchemist,454,286


(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

- Hemos cargado los dos archivos con información de usuarios y de los artistas que estos usuarios han escuchado alguna vez.  De acuerdo con el archivo `readme.txt` que describe la base, encontramos que el separador es tabulador, por tanto, tuvimos en cuenta este separador para cargar correctamente las columnas en un *DataFrame* de *pandas*.

- Posteriormente, realizamos las transformaciones a los nombres de las columnas según lo sugerido en el enunciado para facilitar la unión de los dos dataframes por el identificador del usuario.

- Luego, aplicamos el método *merge* para unir los dos dataframes, obteniendo uno solo de **92.834 filas** y **4 columnas**.

### 2. Análisis preliminar. 

En esta sección exploraremos la base. Para ello responda las siguientes preguntas.

#### 2.1 ¿Cuantos usuarios y artistas hay en la base?


In [4]:
f"{users_artists['userID'].nunique()} 🧑 | {users_artists['artistID'].nunique()} 🎵"

'1892 🧑 | 17632 🎵'

Contamos con 1.892 usuarios y 17.632 artistas


#### 2.2 ¿Cuáles es la distribución de probabilidad del consumo por artista? (haga el calculo sin ponderar y ponderando por el numero de reproducciones) ¿Qué podemos inferir a partir de la comparación de ambas?

Esta parte el interés es poder analizar qué tanto se escucha cada artista.  Consideraremos:

- Un primer escenario (sin ponderar) en el que todos los artistas son igualmente relevantes sin tener en cuenta el número de reproducciones, únicamente cuántos usuarios lo escucharon.
- Un segundo escenario (ponderando) en el que los artistas adquieren cierto peso o relevancia según la cantidad de reproducciones que han tenido entre los usuarios.

In [101]:
# Utilice este espacio para escribir el código.
nbins = users_artists['artistID'].nunique()

# Sin ponderar
# Solo tenemos en cuenta cuántos usuarios han "consumido" cada artista.  Y lo usamos para obtener la probabilidad de que cada artista pueda ser escuchado del total de artistas.
artists_stats = users_artists.groupby('artistID')['userID'].nunique().reset_index().rename(columns={'userID': 'usuarios_escuchado'})
artists_stats['puntaje_no_ponderado'] = artists_stats['usuarios_escuchado'] / users_artists['artistID'].nunique()
artists_stats.sort_values(by='puntaje_no_ponderado', ascending=False)
artists_stats.set_index('artistID', inplace=True)

# Para ponderar las reproducciones por artista sumamos las reproducciones por artista.
# De esta manera las reproducciones de cada artista tendrán una relevancia dentro de la probabilidad de que ese artista sea escuchado del total de reproducciones.
total_reproductions = users_artists['nro_reproducciones'].sum()
artists_stats['nro_reproducciones'] = users_artists.groupby('artistID').agg({'userID': 'nunique', 'nro_reproducciones': 'sum'})['nro_reproducciones']
artists_stats['puntaje_ponderado'] = users_artists.groupby('artistID').agg({'userID': 'nunique', 'nro_reproducciones': 'sum'})['nro_reproducciones'] / total_reproductions
artists_stats.reset_index(inplace=True)

fig_sin_ponderar = px.histogram(artists_stats, x='artistID', y='puntaje_no_ponderado', nbins=nbins, title='Distribución de probabilidad sin ponderar del consumo por artista')
fig_ponderada = px.histogram(artists_stats, x='artistID', y='puntaje_ponderado', nbins=nbins, title='Distribución de probabilidad ponderada del consumo por artista')

# Agregamos los nombres de los artistas a nuestro dataframe de estadísticas para facilitar visualizaciones más adelante
artists_stats.set_index('artistID')
artists_stats = pd.merge(artists, artists_stats, on='artistID')

# Para mostrar los dos gráficos juntos...
fig = sp.make_subplots(rows=1, cols=2)
fig.add_trace(fig_sin_ponderar['data'][0], row=1, col=1)
fig.add_trace(fig_ponderada['data'][0], row=1, col=2)

# Personalizar etiquetas de ejes
fig.update_xaxes(title_text='artistID', row=1, col=1)
fig.update_xaxes(title_text='artistID', row=1, col=2)
fig.update_yaxes(title_text='Probabilidad que el artista sea escuchado', row=1, col=1)
fig.update_yaxes(title_text='Probabilidad que haya una reproducción del artista', row=1, col=2)
fig.update_layout(title_text='Distribución del consumo por artista (Sin ponderar reproducciones vs. Reproducciones ponderadas)', showlegend=False)

fig.show()

In [102]:
# Evaluemos una de las mayores diferencias entre 2 artistas para los que la probabilidad de ser escuchado cambió
artists_stats[artists_stats['artistID'].isin([89, 289])]

,artistID,artistName,usuarios_escuchado,puntaje_no_ponderado,nro_reproducciones,puntaje_ponderado
83,89,Lady Gaga,611,0.034653,1291387,0.018666
283,289,Britney Spears,522,0.029605,2393140,0.034591


(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

- Hayamos la probabilidad de que un artista sea escuchado del total de artistas existentes claculando previamente la cantidad de usuarios que han escuchado a cada artista.
- Luego hayamos la probabilidad de que un artista sea reproducido del total del reproducciones registradas, de esta manera, tenemos en cuenta el peso o popularidad de cada artista de acuerdo a su cantidad de reproducciones.
- Generámos gráficas en plotly para poder comparar los dos gráficos de una forma más cómoda.

- Qué podemos inferir a partir de la comparación de ambas?
  - No ponderar el consumo de los artistas resulta ser un escenario más equitativo para los artistas pues todos serán considerados con la misma importancia sin ser relevante si han sido escuchados 1 o 1 millón de veces, pero del lado del usuario, esto podría resultar en recomendaciones no acordes a los gustos del usuario.
  - Ponderar el consumo de los artistas dando un peso a cada uno según la cantidad de reproducciones, dará un indicio de su popularidad.  Esto permitirá identificar los artistas más influyentes y darles prioridad a esos artistas en una futura recomendación.
  - Como existen más de 17 mil artistas no es fácil ver las diferencias.  Pero para entender un poco mejor el escenario, tomamos 2 de los artistas más populares, Lady Gaga y Britney Spears.  Bajo el primer escenario Lady Gaga resulta más relevante sobre Britney Spears puesto que más usuarios han escuchado a Lady.  En el segundo escenario, Britney resulta más popular de las 2 porque es quien ha tenido mayor cantidad de reproducciones.

#### 2.3 Para el usuario 8 (`userID==8`) ¿cuál es la distribución de reproducción de artistas basado en el número de reproducciones relativas?. Presente sus resultados usando tablas y/o gráficas. ¿Encuentra algún patrón en los artistas que escucha y las veces que reproduce? ¿Podemos decir algo de sus preferencias?


In [88]:
# Utilice este espacio para escribir el código.
user_data = users_artists[users_artists['userID'] == 8]

fig = px.histogram(user_data.sort_values(by='nro_reproducciones', ascending=False), x='artistName', y='nro_reproducciones', title=f'Reproducciones por artista para usuario 8')
fig.show()

In [89]:
# Extraemos el top 10 de artistas más escuchados por el usuario 8.
user_8_ordered_artists = user_data.sort_values(by='nro_reproducciones', ascending=False)
user_8_ordered_artists.head(10)

,artistID,artistName,userID,nro_reproducciones
19523,334,No Angels,8,6291
19528,335,Sandy,8,2396
13176,289,Britney Spears,8,2258
19530,336,Vanessa Petruo,8,1735
14742,295,Beyoncé,8,1425
19531,337,Alexander,8,1407
15139,296,Sugababes,8,1039
19532,338,Queensberry,8,963
3294,89,Lady Gaga,8,921
19534,339,Fady Maalouf,8,819


(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

- Filtramos la base unificada de artistas particularmente para el usuario 8.
- Ordenamos los resultados por el número de reproducciones para cada artista y los graficamos en un histograma.

- A partir del histograma, encontramos que el usuario 8 ha escuchado a 50 artistas.  Dentro de los 10 artistas que más ha reproducido este usuario, podemos ver algunos artístas reconocidos que están asociados al género pop, realizando una consulta externa encontramos que los otros artistas no tan conocidos también se encuentran asociados a alguna variante del género pop, como electro pop o pop rock.  En nuestro dataset no disponemos de un atributo que indique el género o generos musicales asociados al artista, por tanto, si deseamos realizar recomendaciones para este usuario podemos recurrir al filtrado colaborativo, es decir, encontrar recomendaciones basado en las preferencias de usuarios que más se parezcan al usuario 8.

### 3. Generando Recomendaciones

En esta sección nos interesa generar recomendaciones ***nuevas y relevantes*** para el usuario 8 (`userID==8`). Para ello vamos a generar distintos sistemas de recomendación y comparar las recomendaciones generadas.

#### 3.1. Filtrado colaborativo sencillo: promedios simples.

Usando el promedio simple de reproducciones (sin considerar el número de reproducciones) genere una tabla y/o gráfica con 10 recomendaciones de artistas para este usuario. Explique con cuidado su procedimiento y justifique sus elecciones.

In [109]:
# Utilice este espacio para escribir el código.
# Creamos un nuevo dataframe solo con estadísticas de los artistas que el usuario 8 no ha escuchado aún.
user_8_new_artists = artists_stats[~artists_stats['artistID'].isin(user_8_ordered_artists['artistID'])]

# Ordenamos descendentemente este resultado por el puntaje no ponderado obtenido en el punto 2.2
user_8_recomm_1 = user_8_new_artists.sort_values(by='puntaje_no_ponderado', ascending=False)

# Obtenemos los 10 artistas con mejor puntuación no ponderada y mostramos sus nombres
user_8_recomm_1_artists = user_8_recomm_1.head(10)['artistName']
print(f"Hey usuario 8!, tal vez también quieras escuchar... {', '.join(user_8_recomm_1_artists)}")
user_8_recomm_1.head(10)

Hey usuario 8!, tal vez también quieras escuchar... The Beatles, Madonna, Avril Lavigne, Muse, Paramore, Radiohead, Coldplay, Ke$ha, Shakira, Black Eyed Peas


,artistID,artistName,usuarios_escuchado,puntaje_no_ponderado,nro_reproducciones,puntaje_ponderado
221,227,The Beatles,480,0.027223,662116,0.009570
61,67,Madonna,429,0.024331,921198,0.013315
327,333,Avril Lavigne,417,0.023650,525844,0.007601
184,190,Muse,400,0.022686,485076,0.007011
492,498,Paramore,399,0.022629,963449,0.013926
148,154,Radiohead,393,0.022289,385306,0.005569
59,65,Coldplay,369,0.020928,330757,0.004781
460,466,Ke$ha,362,0.020531,384405,0.005556
695,701,Shakira,319,0.018092,688529,0.009952
300,306,Black Eyed Peas,304,0.017241,188634,0.002727


(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

- Identificamos los artistas que el usuario ya ha reproducido y los excluimos de nuestro dataframe unificado de artistas por usuario. De esta manera evitamos recomendar algún artista que el usuario ya haya escuchado.
- Como ya contábamos con la puntuación no ponderada por artista, ordenamos el resultado del paso anterior de manera descendente para obtener los 10 mejores artistas a recomendar
- Posteriormente, mostramos los nombres de esos 10 artistas

- La técnica de filtrado colaborativo sencillo con el promedio simple nos permitió identificar aquellos artistas que también han sido muy escuchados por otros usuarios que aún el usuario 8 no había escuchado.  Esta técnica puede ser adecuada si consideramos que recomendar solo nuevos artistas es lo que deseamos para nuestros usuarios.  Hay que tener en cuenta que esta técnica es sensible a aquellos casos en los que varios usuarios han escuchado un artista poco conocido o que podrían no ser de interés del usuario 8, el promedio o puntuación no ponderada hallada en el paso anterior de cierta medida está influenciada por este tipo de escenarios que no reflejan gustos particulares.

#### 3.2.  Filtrado colaborativo sencillo: promedios ponderados.

Usando el promedio de reproducciones ponderado por `nro_reproducciones` genere una tabla y/o gráfica con 10 recomendaciones de artistas para este usuario. Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en el paso anterior.


In [110]:
# Utilice este espacio para escribir el código.
# Ordenamos descendentemente este resultado por el puntaje ponderado obtenido en el punto 2.2
user_8_recomm_2 = user_8_new_artists.sort_values(by='puntaje_ponderado', ascending=False)

# Obtenemos los 10 artistas con mejor puntuación ponderada y mostramos sus nombres
user_8_recomm_2_artists = user_8_recomm_2.head(10)['artistName']
print(f"Hey usuario 8!, tal vez también quieras escuchar... {', '.join(user_8_recomm_2_artists)}")
user_8_recomm_2.head(10)

Hey usuario 8!, tal vez también quieras escuchar... Depeche Mode, Paramore, Madonna, Shakira, The Beatles, Avril Lavigne, Evanescence, Glee Cast, U2, Miley Cyrus


,artistID,artistName,usuarios_escuchado,puntaje_no_ponderado,nro_reproducciones,puntaje_ponderado
66,72,Depeche Mode,282,0.015994,1301308,0.018809
492,498,Paramore,399,0.022629,963449,0.013926
61,67,Madonna,429,0.024331,921198,0.013315
695,701,Shakira,319,0.018092,688529,0.009952
221,227,The Beatles,480,0.027223,662116,0.009570
327,333,Avril Lavigne,417,0.023650,525844,0.007601
372,378,Evanescence,226,0.012818,513476,0.007422
673,679,Glee Cast,249,0.014122,506453,0.007320
505,511,U2,185,0.010492,493024,0.007126
455,461,Miley Cyrus,286,0.016221,489065,0.007069


(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

- Ya conocíamos los artistas que el usuario 8 no había escuchado aún, por tanto en este paso ordenamos por la puntuación ponderada de manera descendente para obtener los 10 mejores artistas a recomendar según ese criterio y los mostramos junto con los nombres de los artistas

- La técnica de filtrado colaborativo sencillo con el promedio ponderado nos permitió identificar aquellos artistas con mayores reproducciones que aún el usuario 8 no había escuchado.  Hay que tener en cuenta que esta técnica es sensible a aquellos artistas que han tenido pocas canciones, pero esas pocas han sido un éxito y por su cantidad de reproducciones han desplazado a otros artistas, por ejemplo, Glee Cast, dado que goza de una buena puntuación a raíz de sus más de 500 mil reproducciones se muestra como recomendado a pesar de no ser tan reconocido.

#### 3.3.  Filtrado colaborativo sencillo: similitud de coseno.

Usando el promedio ponderado de reproducciones genere una tabla y/o gráfica  con 10 recomendaciones de artistas para este usuario. Para generar los pesos utilice la distancia de coseno. Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en el paso anterior.

In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

#### 3.4.  Filtrado colaborativo usando SVD


Usando la descomposición en valores singulares (SVD) genere una tabla y/o gráfica  con 10 recomendaciones de artistas para este usuario.  Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en los pasos anteriores.


In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

#### 3.5.  Filtrado colaborativo usando Análisis de Canasta de Compra

Usando  el algoritmo `Apriori` genere una tabla y/o gráfica  con 10 recomendaciones de artistas para este usuario.  Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en los pasos anteriores. Esto puede tomar mucho tiempo, sea cuidadoso al elegir los hiper-parámetors del modelo, utilice los resultados de las estadísticas descriptivas para elegir sus hier-parámetros, y genere solo reglas con 2 elementos. (Puede también aprovechar los recursos de [Google Colab](https://colab.research.google.com/))


In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

### 4. Recomendaciones generales 

De acuerdo con los resultados encontrados, en su opinión ¿qué procedimiento generó las mejores recomendaciones para este usuario? ¿Cómo implementaría una evaluación objetiva de estas recomendaciones? Justifique su respuesta.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).